# Gaussian Naive Bayes Classification

In [1]:
from sklearn.datasets import  load_iris

In [2]:
iris = load_iris()
data =  iris.data
target = iris.target

In [3]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3)

In [5]:
from sklearn.naive_bayes import GaussianNB

In [6]:
model = GaussianNB()

In [7]:
model.fit(X_train, y_train)

GaussianNB()

In [15]:
predictions = model.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score

In [19]:
accuracy = accuracy_score(y_test, predictions)

In [21]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9777777777777777


# Hyperparameter Tuning using GridSearchCV

In [28]:
from sklearn.model_selection import GridSearchCV
import numpy as np

In [30]:
param_grid = {
    'var_smoothing': np.logspace(-10, -1, 10)  # Values between 1e-10 to 1e-1
}

In [32]:
grid_search = GridSearchCV(model, param_grid, cv=2)

In [34]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=GaussianNB(),
             param_grid={'var_smoothing': array([1.e-10, 1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03,
       1.e-02, 1.e-01])})

In [36]:
grid_search.best_params_

{'var_smoothing': 0.01}

In [38]:
best_model = grid_search.best_estimator_

In [40]:
best_model.fit(X_train, y_train)

GaussianNB(var_smoothing=0.01)

In [42]:
predictions = best_model.predict(X_test)

In [44]:
accuracy = accuracy_score(y_test, predictions)

In [46]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9777777777777777


# Hyperparameter Tuning using Optuna

In [49]:
import optuna

In [51]:
# Define objective function for Optuna
def objective(trial):
    var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-10, 1e-1)

    model = GaussianNB(var_smoothing=var_smoothing)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    Accuracy = accuracy_score(y_test, y_pred)
    return Accuracy

In [53]:
# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-03-01 14:53:19,636] A new study created in memory with name: no-name-10c6e6aa-d300-4e74-b2a2-cb983668bead
C:\Users\suren\AppData\Local\Temp\ipykernel_3756\4054314264.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-10, 1e-1)
[I 2025-03-01 14:53:19,642] Trial 0 finished with value: 0.9777777777777777 and parameters: {'var_smoothing': 1.1852863950140933e-08}. Best is trial 0 with value: 0.9777777777777777.
C:\Users\suren\AppData\Local\Temp\ipykernel_3756\4054314264.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-10, 1e-1)
[I 2025-03-01 1

In [55]:
study.best_params

{'var_smoothing': 0.06867747308097656}

In [57]:
study.best_value

1.0

In [59]:
best_params = study.best_params

In [61]:
best_model1 = GaussianNB(**best_params)

In [63]:
best_model1.fit(X_train, y_train)

GaussianNB(var_smoothing=0.06867747308097656)

In [65]:
predictions = best_model1.predict(X_test)

In [67]:
accuracy = accuracy_score(y_test, predictions)

In [69]:
print(f"Accuracy: {accuracy}")

Accuracy: 1.0
